In [6]:
from tensorflow.keras.utils import plot_model

In [8]:
plot_model(model_full)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [2]:
# Imports
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from city_sustainability.quality import life_quality
from city_sustainability.models import unet_model
from tensorflow.keras.callbacks import EarlyStopping
from city_sustainability.preprocessing import image_resize
from city_sustainability.loading_data_448x448 import image_and_label_arrays_448
from city_sustainability.loading_data_448x448 import loading_paths_448

2023-05-28 16:16:42.719041: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 16:16:42.741898: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 16:16:42.742551: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 16:16:43.283662: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, UpSampling2D, Concatenate, Input


def encoder(inputs):
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Dropout(0.1)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Dropout(0.1)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    return conv1, conv2, pool2


def decoder(conv1, conv2, encoded):
    up1 = UpSampling2D(size=(2, 2))(encoded)
    merge1 = Concatenate(axis=3)([conv2, up1])
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge1)
    conv3 = BatchNormalization()(conv3)
    conv3 = Dropout(0.1)(conv3)

    up2 = UpSampling2D(size=(2, 2))(conv3)
    merge2 = Concatenate(axis=3)([conv1, up2])
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge2)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.1)(conv4)

    return conv4


###########################


def decoder_full(conv1, conv2, conv3, conv4, conv5, encoded):
    
    up5 = UpSampling2D(size=(2, 2))(encoded)
    merge5 = Concatenate(axis=3)([conv5, up5])
    l5 = Conv2D(512, (3, 3), activation='relu', padding='same')(merge5)
    l5 = BatchNormalization()(l5)
    l5 = Dropout(0.1)(l5) 
    
    up4 = UpSampling2D(size=(2, 2))(l5)
    merge4 = Concatenate(axis=3)([conv4, up4])
    l4 = Conv2D(512, (3, 3), activation='relu', padding='same')(merge4)
    l4 = BatchNormalization()(l4)
    l4 = Dropout(0.1)(l4) 
    
    up3 = UpSampling2D(size=(2, 2))(l4)
    merge3 = Concatenate(axis=3)([conv3, up3])
    l3 = Conv2D(256, (3, 3), activation='relu', padding='same')(merge3)
    l3 = BatchNormalization()(l3)
    l3 = Dropout(0.1)(l3)   
    
    up2 = UpSampling2D(size=(2, 2))(l3)
    merge2 = Concatenate(axis=3)([conv2, up2])
    l2 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge2)
    l2 = BatchNormalization()(l2)
    l2 = Dropout(0.1)(l2)

    up1 = UpSampling2D(size=(2, 2))(l2)
    merge1 = Concatenate(axis=3)([conv1, up1])
    l1 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge1)
    l1 = BatchNormalization()(l1)
    l1 = Dropout(0.1)(l1)

    return l1


#############################


def build_model(input_shape=(28, 28, 1), num_classes=12):
    inputs = Input(input_shape)
    conv1, conv2, encoded = encoder(inputs)
    decoded = decoder(conv1, conv2, encoded)
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(decoded)
    return Model(inputs=inputs, outputs=outputs)


def build_vgg16_model(input_shape=(28, 28, 1), num_classes=12 ):
    model_vgg = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
    for layer in model_vgg.layers:
        layer.trainable = False
    inputs = model_vgg.layers[0].output
    conv1 = model_vgg.get_layer('block1_conv2').output
    conv2 = model_vgg.get_layer('block2_conv2').output
    encoded = model_vgg.get_layer('block3_conv3').output
    decoded = decoder(conv1, conv2, encoded)
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(decoded)
    return Model(inputs=inputs, outputs=outputs)


#############################

def build_vgg16_model_full(input_shape=(28, 28, 1), num_classes=12):
    model_vgg = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
    for layer in model_vgg.layers:
        layer.trainable = False
    inputs = model_vgg.layers[0].output
    conv1 = model_vgg.get_layer('block1_conv2').output
    conv2 = model_vgg.get_layer('block2_conv2').output
    conv3 = model_vgg.get_layer('block3_conv3').output
    conv4 = model_vgg.get_layer('block4_conv3').output
    conv5 = model_vgg.get_layer('block5_conv3').output
    encoded = model_vgg.get_layer('block5_pool').output       
    decoded = decoder_full(conv1, conv2, conv3, conv4, conv5, encoded)   
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(decoded)    
    return Model(inputs=inputs, outputs=outputs)


#############################


def compile_model(model, optimizer='adam'):
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


def compute_iou(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true + y_pred, axis=[1, 2, 3]) - intersection
    iou = tf.reduce_mean((intersection + 1e-7) / (union + 1e-7))
    return iou


def train_model(model, x, y, epochs=1, batch_size=32, validation_split=0.1):
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy', compute_iou])
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001, verbose=1)
    early_stopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
    history = model.fit(x, y, epochs=epochs, batch_size=batch_size, validation_split=validation_split, callbacks=[lr_reducer, early_stopper])
    return history


def evaluate_model(model, x, y):
    loss, accuracy = model.evaluate(x, y)
    
    # Calculate IoU
    y_pred = model.predict(x)
    iou = compute_iou(y, y_pred)
    
    print("Evaluation results:")
    print(f"Loss: {loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"IoU: {iou:.4f}")


def predict(model, x):
    return model.predict(x)


In [10]:
import numpy as np

In [14]:

# Retrieve image_paths and label_paths
data_path_1 = '../../raw_data/OpenEarthMap_wo_xBD'
image_paths_448, label_paths_448 = loading_paths_448(data_path_1)
# Import all image array as X and label array as y
X, y = image_and_label_arrays_448(image_paths_448, label_paths_448, sampling_ratio=0.01)
X

array([[[[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [135, 143, 104],
         [108, 115,  88],
         [ 80,  75,  72]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [117, 108,  89],
         [ 78,  74,  69],
         [101,  96,  85]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [140, 136, 109],
         [144, 132, 112],
         [151, 137, 117]],

        ...,

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [154, 137, 119],
         [ 69,  69,  71],
         [ 26,  36,  47]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [130, 120, 105],
         [151, 137, 120],
         [ 89,  85,  82]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [152, 135, 116],
        

In [5]:
X_test_scaled.shape

(9, 448, 448, 3)

In [4]:
# Create the model
model_full = build_vgg16_model_full(input_shape=(448, 448, 3), num_classes = 9)

# When running from .py
# model_full = unet_model.build_vgg16_model_full(input_shape=(400, 400, 3), num_classes = 9)

2023-05-26 13:54:12.328681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-26 13:54:12.331687: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
# Compile the model

unet_model.compile_model(model_full)

In [ ]:
model_full

In [ ]:
model_full.summary()

In [ ]:
unet_model.train_model(model_full, X_train_scaled, y_train, epochs = 1, batch_size=18, validation_split=0.2)

In [ ]:
#Load VGG16 model
model_vgg = VGG16(weights="imagenet", include_top=False, input_shape=(256,256,3))

In [ ]:
#What is inside VGG16 model
model_vgg.summary()